In [1]:
import os
import tensorflow as tf
import numpy as np
from tqdm import tqdm_notebook as tqdm
import elm
from f3_cnn import CNN
import matplotlib.pyplot as plt
from sklearn import preprocessing
import scipy
import scipy.io
import pickle

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
DEVICE = '/device:GPU:0'
EPOCHS = 50
BATCH_SIZE = 16
w = 1       
SAVE_PATH = 'backup/'

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
def load(fromIndex, toIndex):
    datas = None
    labels = None
    for i in range(1, 14):
        print(i)
        for t in range(1, 3):
            for index in range(fromIndex, toIndex):
                fileperfix = 'data/SEED1/feature3_imgs/person' + str(i) + '_time' + str(t) + '_' + str(index)
                data = np.load(fileperfix + '.npy')
                label = np.load(fileperfix + '_label.npy')
                if datas is None:
                    datas = data
                    labels = label
                else:
                    datas = np.concatenate((datas, data))
                    labels = np.concatenate((labels, label))
    return datas, labels

def dense_to_one_hot(labels_dense, num_classes = 3):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

In [ ]:
d, l = load(1, 15)

In [ ]:
permutation = np.random.permutation(d.shape[0])
x_train = d[permutation, :, :, :3]
y_train = l[permutation]
y_train = dense_to_one_hot(y_train)

In [ ]:
X = tf.placeholder(tf.float32, [None, 62, 62, 5])
Y = tf.placeholder(tf.float32, [None, 3])
LR = tf.placeholder(tf.float32)

In [ ]:
net = CNN(X, Y, DEVICE)

In [ ]:
optimizer = tf.train.AdamOptimizer(LR).minimize(net.loss)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(Y, 1), tf.argmax(net.logits,1)), dtype=tf.float32))
sess = tf.Session(config=tf.ConfigProto(log_device_placement=False,allow_soft_placement=True))
init = tf.global_variables_initializer()
sess.run(init)
saver = tf.train.Saver()

In [ ]:
allloss = []
with tf.device(DEVICE):
    epoch = 0
    test_acc = []
    while epoch < EPOCHS:
        total_steps = int(x_train.shape[0]/BATCH_SIZE)
        for step in tqdm(range(total_steps), desc=('Epoch '+str(epoch))):  
            if step*BATCH_SIZE + BATCH_SIZE < x_train.shape[0]:
                BATCH_X = x_train[step*BATCH_SIZE: step*BATCH_SIZE+BATCH_SIZE]
                BATCH_Y = y_train[step*BATCH_SIZE: step*BATCH_SIZE+BATCH_SIZE]
            else:
                BATCH_X = x_train[step*BATCH_SIZE:]
                BATCH_Y = y_train[step*BATCH_SIZE:]
    
            [loss_value, _] = sess.run([net.loss, optimizer], feed_dict={X: BATCH_X, LR: 0.01})
        allloss.append(loss_value)
        print("epoch %d: loss %f" % (epoch, loss_value))  
        saver.save(sess, SAVE_PATH+'model.ckpt')
        epoch += 1

In [ ]:
plt.plot(allloss)
plt.show()